<a href="https://colab.research.google.com/github/kbro4/Reliever-Value/blob/main/Getting_bWAR_by_Year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
# Cleans data and gets percentage of team WAR by positional group
def get_shares(df):
  df = df[0].drop(30)
  df = df.apply(lambda x: x.sort_values().reset_index(drop=True))

  # Total
  df['Team'] = df['Total'].str.split(r'(-?\d+)', expand=True)[0]
  df['Team Sum1'] = df['Total'].str.split(r'(-?\d+)', expand=True)[1]
  df['Team Sum2'] = df['Total'].str.split(r'(-?\d+)', expand=True)[2]
  df['Team Sum3'] = df['Total'].str.split(r'(-?\d+)', expand=True)[3]

  # Starting Pitchers
  df['SP Team'] = df['SP'].str.split(r'(-?\d+)', expand=True)[0]
  df['SP Sum1'] = df['SP'].str.split(r'(-?\d+)', expand=True)[1]
  df['SP Sum2'] = df['SP'].str.split(r'(-?\d+)', expand=True)[2]
  df['SP Sum3'] = df['SP'].str.split(r'(-?\d+)', expand=True)[3]

  # Relief Pitchers
  df['RP Team'] = df['RP'].str.split(r'(-?\d+)', expand=True)[0]
  df['RP Sum1'] = df['RP'].str.split(r'(-?\d+)', expand=True)[1]
  df['RP Sum2'] = df['RP'].str.split(r'(-?\d+)', expand=True)[2]
  df['RP Sum3'] = df['RP'].str.split(r'(-?\d+)', expand=True)[3]

  # Position Players
  df['Pos Team'] = df['Non-P'].str.split(r'(-?\d+)', expand=True)[0]
  df['Pos Sum1'] = df['Non-P'].str.split(r'(-?\d+)', expand=True)[1]
  df['Pos Sum2'] = df['Non-P'].str.split(r'(-?\d+)', expand=True)[2]
  df['Pos Sum3'] = df['Non-P'].str.split(r'(-?\d+)', expand=True)[3]

  # Total
  df['Team Sum'] = df[['Team Sum1', 'Team Sum2', 'Team Sum3']].agg(' '.join, axis=1)
  df['Team Sum'] = df['Team Sum'].str.replace(' ', '').astype(float)

  # Starting Pitchers
  df['SP Sum'] = df[['SP Sum1', 'SP Sum2', 'SP Sum3']].agg(' '.join, axis=1)
  df['SP Sum'] = df['SP Sum'].str.replace(' ', '').astype(float)

  # Relief Pitchers
  df['RP Sum'] = df[['RP Sum1', 'RP Sum2', 'RP Sum3']].agg(' '.join, axis=1)
  df['RP Sum'] = df['RP Sum'].str.replace(' ', '').astype(float)

  # Position Players
  df['Pos Sum'] = df[['Pos Sum1', 'Pos Sum2', 'Pos Sum3']].agg(' '.join, axis=1)
  df['Pos Sum'] = df['Pos Sum'].str.replace(' ', '').astype(float)

  needed_columns = ['Team', 'Team Sum', 'SP Team', 'SP Sum', 'RP Team', 'RP Sum', 'Pos Team', 'Pos Sum']
  new_df = df[needed_columns]

  new_df['SP Share'] = new_df['SP Sum'] / new_df['Team Sum']
  new_df['RP Share'] = new_df['RP Sum'] / new_df['Team Sum']
  new_df['Pos Share'] = new_df['Pos Sum'] / new_df['Team Sum']

  return new_df

In [ ]:
df_list = []

# Iterates through years, scrapes bWAR data from baseball reference, and appends it to dataframe
for i in range(2013, 2024):
  if i != 2020:
    url = f'https://www.baseball-reference.com/leagues/team_compare.cgi?request=1&year={i}&lg=MLB&stat=WAR'
    df = pd.read_html(url)
    df_list.append(get_shares(df))


# Specify the Excel file path
excel_file_path = 'output.xlsx'

# Use ExcelWriter to write bWAR data to excel file
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    for i, df in enumerate(df_list, start=1):
        sheet_name = f'Sheet{i}'  # Create sheet names dynamically
        df.to_excel(writer, sheet_name=sheet_name, index=False)

<ipython-input-2-8db768b69096>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['SP Share'] = new_df['SP Sum'] / new_df['Team Sum']
<ipython-input-2-8db768b69096>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['RP Share'] = new_df['RP Sum'] / new_df['Team Sum']
<ipython-input-2-8db768b69096>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa